In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils import pattern_utils

In [14]:
class modern_hopfield():
  
  func_type = "Polyn"
  def __init__(self, N: int):
    """
    `N`: number of neurons of the network
    """
    self.N = N
    self.memory_matrix = np.zeros(shape=(1,N))
    return
  
  def add_pattern(self, p: np.ndarray):
    """
    `p`: pattern to add to memory
    """
    if self.memory_matrix[0,0]==0:
      self.memory_matrix = p
    else:
      self.memory_matrix = np.hstack((self.memory_matrix, p))
    return

  def retrieve_pattern(self, p: np.ndarray, epochs: int):
    """
    `p`: input pattern  \n
    `epochs`: maximum number of iterations \n
    performs the update rule up to `epochs` times, trying to match the input pattern `p` to a pattern stored in memory     
    """
    sigma = p.copy()
    for epoch in tqdm(range(epochs)):
      sigma = self._update_rule(sigma)
    return sigma

  def _update_rule(self, sigma: np.ndarray):
    """
    `sigma`: vector containing the values of the network's neurons at time t. \n 
    The update rule returns the values of the network's neurons at time t+1 
    """
    s = 0
    for xi in self.memory_matrix:
      s += self._F(xi + np.dot(xi, sigma) - np.multiply(xi, sigma)) - self._F(-xi + np.dot(xi, sigma) - np.multiply(xi, sigma))
    return np.sign(s)
  
  n=3
  def _F(self, x):
    """
    Energy function used in the update rule. \n
    Different types of function:
    - `Polyn`: polynomial function of order n
    - `RePn`: rectified polynomial function of order n
    """
    match self.func_type:
      case "Polyn":
        return x**self.n
      case "RePn":
        return np.maximum(0, x**self.n)
  


In [15]:
pattern_lenght = 100
n_patterns = 100

model = modern_hopfield(pattern_lenght)

In [16]:
patterns = pattern_utils.generate_patterns(pattern_lenght=pattern_lenght, N=n_patterns)
model.add_pattern(patterns)

In [22]:
corrupted = pattern_utils.corrupt_patterns(patterns=patterns, q=1, corruption_type="Erase")
np.array_equal(corrupted, patterns)
corrupted

array([[-1,  1, -1, ...,  1,  1, -1],
       [-1,  1,  1, ...,  1, -1,  1],
       [ 1, -1, -1, ..., -1,  1, -1],
       ...,
       [ 1,  1, -1, ...,  1, -1,  1],
       [-1, -1, -1, ...,  1,  1, -1],
       [-1, -1, -1, ...,  1,  1,  1]])

In [21]:
print(model.memory_matrix[0,:], "\n")
print(corrupted[0,:])

[-1  1 -1 -1  1 -1  1  1 -1  1 -1 -1  1 -1 -1  1 -1 -1  1  1 -1  1  1 -1
  1  1  1 -1 -1  1  1  1  1  1 -1 -1  1  1  1  1 -1 -1  1  1 -1 -1  1 -1
 -1  1 -1 -1  1 -1 -1 -1 -1 -1  1  1  1  1  1  1  1  1 -1  1 -1  1 -1 -1
 -1 -1 -1  1 -1  1  1 -1 -1 -1  1 -1  1 -1  1 -1  1 -1  1  1 -1 -1  1 -1
 -1  1  1 -1] 

[-1  1 -1 -1  1 -1  1  1 -1  1 -1 -1  1 -1 -1  1 -1 -1  1  1 -1  1  1 -1
  1  1  1 -1 -1  1  1  1  1  1 -1 -1  1  1  1  1 -1 -1  1  1 -1 -1  1 -1
 -1  1 -1 -1  1 -1 -1 -1 -1 -1  1  1  1  1  1  1  1  1 -1  1 -1  1  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]


In [18]:
for i, c in enumerate(corrupted):
  r = model.retrieve_pattern(c, epochs=100)
  if np.array_equal(r, patterns[i]): print(f"Pattern #{i} recognized")
  else: print(f"Pattern #{i} not recognized")

100%|██████████| 100/100 [00:00<00:00, 679.14it/s]


Pattern #0 recognized


100%|██████████| 100/100 [00:00<00:00, 716.43it/s]


Pattern #1 recognized


100%|██████████| 100/100 [00:00<00:00, 504.37it/s]


...

Pattern #97 recognized


100%|██████████| 100/100 [00:00<00:00, 1606.73it/s]


Pattern #98 recognized


100%|██████████| 100/100 [00:00<00:00, 1539.89it/s]

Pattern #99 recognized
